In [1]:
#import usefull modules
import pandas as pd
import scipy.stats as stats
import numpy as np
from functions import cleanup
from pprint import pprint

In [2]:
#import files and read the data
path_2019 = "../../data/team_player/raw/players_20.csv"
path_2020 = "../../data/team_player/raw/players_20.csv"
path_epl = "../../data/team_player/cleaned/EPL.csv"
path_liga = "../../data/team_player/cleaned/Liga.csv"
path_seriea = "../../data/team_player/cleaned/SerieA.csv"

players_19 = pd.read_csv(path_2019)
players_20 = pd.read_csv(path_2020)
epl_df = pd.read_csv(path_epl)
liga_df = pd.read_csv(path_liga)
seriea_df = pd.read_csv(path_seriea)
list_leagues = [epl_df,liga_df,seriea_df]

In [3]:
players_19 = players_19.loc[:,"short_name":'goalkeeping_reflexes'].rename({"club": "Teams"},axis=1)
players_20 = players_20.loc[:,"short_name":'goalkeeping_reflexes'].rename({"club": "Teams"},axis=1)
del players_19["dob"]
del players_19["long_name"]
del players_20["dob"]
del players_20["long_name"]
players_19 = cleanup(players_19)
players_20 = cleanup(players_20)

In [4]:
#We want to know a bit more about the players in the top 4 of each team and averages per position
players_19 = players_19[(players_19["Teams"].isin(epl_df["Teams"])) | (
    players_19["Teams"].isin(liga_df["Teams"]) | players_19["Teams"].isin(seriea_df["Teams"]))]
players_20 = players_20[(players_20["Teams"].isin(epl_df["Teams"])) | (
    players_20["Teams"].isin(liga_df["Teams"]) | players_20["Teams"].isin(seriea_df["Teams"]))]

In [5]:
# position groups
list_players = [players_19,players_20]
strikers = ["^LF$","^LS$","^CF$","^RS$","^RF$"]
middfilders_off = ["^LAM$","^RAM$"]
middfilders_central = ["^M$","^LM$","^RM$","^RCM$",]
middfilders_def = ["^DM$"]
central_back = ["^CH$","^RCB$"]
left_back = ["^LH$","^LWD$","^LCB$","^LDM$","^LWB$"]
right_back = ["^RH$","^RWD$","^RCB$","^RDM$","^RWB$"]
goalkeeper = ["^SUB$","^RES$"]
for df in list_players:
    for position in strikers:
        df["team_position"] = df["team_position"].str.replace(position,"ST")
    for position in middfilders_off:
        df["team_position"] = df["team_position"].str.replace(position,"CAM")
    for position in middfilders_central:
        df["team_position"] = df["team_position"].str.replace(position,"CM")
    for position in middfilders_def:
        df["team_position"] = df["team_position"].str.replace(position,"CDM")
    for position in central_back:
        df["team_position"] = df["team_position"].str.replace(position,"CB")
    for position in left_back:
        df["team_position"] = df["team_position"].str.replace(position,"LB")
    for position in right_back:
        df["team_position"] = df["team_position"].str.replace(position,"RB")
    for position in goalkeeper:
        df["team_position"] = df["team_position"].str.replace(position,"GK")

In [6]:
grouped_df_19 = players_19.groupby("team_position")
grouped_mean_19 = grouped_df_19.mean().reset_index()
grouped_mean_19.head()

,team_position,age,height_cm,weight_kg,overall,potential,value_eur,wage_eur,international_reputation,weak_foot,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,CAM,26.031250,177.906250,70.906250,75.218750,78.875000,1.092656e+07,35250.000000,1.625000,3.250000,...,65.468750,72.375000,50.906250,50.968750,47.187500,10.000000,11.718750,11.437500,10.531250,10.281250
1,CB,27.833333,187.196078,80.617647,75.823529,78.284314,1.011863e+07,39460.784314,1.509804,2.960784,...,43.392157,69.039216,75.598039,77.049020,74.852941,10.156863,10.058824,10.431373,9.784314,10.372549
2,CDM,27.774194,181.612903,75.290323,77.258065,79.548387,1.458548e+07,59258.064516,1.709677,3.322581,...,63.000000,74.967742,70.000000,74.193548,70.838710,9.483871,10.580645,10.225806,10.322581,9.677419
3,CM,26.005747,178.356322,72.465517,75.729885,79.080460,1.212672e+07,40028.735632,1.465517,3.298851,...,61.275862,71.925287,57.971264,57.086207,53.448276,10.747126,10.477011,10.545977,10.373563,11.120690
4,GK,24.569966,182.462457,75.841866,69.485779,75.951650,4.747662e+06,23208.191126,1.241752,2.959613,...,48.430603,61.777588,47.340728,47.625711,45.526166,20.102958,19.678043,19.231513,19.782139,20.244596


In [7]:
#we want to know the correlation bewteen the overall rating and the rest of the columns
position_corr = {}
droped_df_2019 = players_19.drop(players_19.columns[[0,4,5,7,8,9,10,25, 26, 21, 22, 23, 24, 18, 19]],axis=1)
for position in list(grouped_mean_19["team_position"].unique()):
    position_corr[position] = droped_df_2019[droped_df_2019["team_position"]== position].corr()["overall"]
position_corr_df = pd.DataFrame.from_dict(position_corr).T 
position_corr_df.head()

,age,height_cm,weight_kg,overall,international_reputation,weak_foot,skill_moves,pace,shooting,passing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
CAM,0.440676,0.050584,0.292400,1.0,0.780899,-0.136690,0.521789,0.091949,0.840953,0.897246,...,0.509283,0.813664,0.259707,0.105429,0.109341,0.243971,-0.426654,-0.083828,0.025025,0.132670
CB,0.125687,0.110979,0.057641,1.0,0.686939,0.188516,0.150873,0.274693,0.349714,0.546229,...,0.260935,0.807685,0.918894,0.956966,0.928167,-0.102771,-0.015970,0.001531,0.144554,0.011186
CDM,0.069401,0.370059,0.041413,1.0,0.788111,-0.004431,0.145858,-0.160148,0.351686,0.695113,...,0.303295,0.787027,0.501659,0.670360,0.611805,-0.045795,-0.226087,-0.049947,-0.159485,-0.075391
CM,0.247221,0.028265,0.079902,1.0,0.724449,0.324863,0.338706,0.037294,0.659328,0.822930,...,0.361926,0.808502,0.231337,0.227791,0.153840,0.109409,0.015622,-0.081033,-0.071744,-0.179638
GK,0.558801,0.121283,0.258060,1.0,0.576240,0.197566,0.265122,0.207283,0.495118,0.678419,...,0.203470,0.704569,0.230310,0.177162,0.151278,0.128324,0.128742,0.121058,0.133187,0.124143


In [8]:
#most correlated values with overall player rating
corr_df = position_corr_df.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=11).iloc[:,1:]
corr_df.head()

,1,2,3,4,5,6,7,8,9,10
CAM,attacking_short_passing,skill_ball_control,passing,skill_dribbling,dribbling,mentality_vision,movement_reactions,shooting,mentality_positioning,mentality_composure
CB,defending,defending_standing_tackle,defending_sliding_tackle,defending_marking,mentality_interceptions,movement_reactions,attacking_heading_accuracy,mentality_composure,international_reputation,mentality_aggression
CDM,movement_reactions,attacking_short_passing,skill_ball_control,mentality_interceptions,international_reputation,skill_long_passing,mentality_composure,mentality_vision,passing,dribbling
CM,skill_ball_control,movement_reactions,passing,attacking_short_passing,mentality_composure,mentality_vision,dribbling,international_reputation,mentality_positioning,skill_dribbling
GK,gk_diving,gk_handling,gk_positioning,gk_reflexes,movement_reactions,gk_kicking,mentality_composure,passing,dribbling,international_reputation


In [9]:
a =players_19[players_19["team_position"]=="CAM"]
a.head()

,short_name,age,height_cm,weight_kg,nationality,Teams,overall,potential,value_eur,wage_eur,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
97,A. Gómez,31,165,68,Argentina,Atalanta,85,85,34000000,92000,...,72,84,50,30,34,8,11,12,9,6
113,D. Alli,23,188,80,England,Tottenham Hotspur,84,89,41500000,115000,...,74,87,70,70,63,7,6,9,11,8
135,M. Özil,30,180,76,Germany,Arsenal,84,84,29500000,125000,...,67,83,24,16,19,6,14,10,6,14
212,R. Nainggolan,31,176,77,Belgium,Cagliari Calcio,83,83,22500000,51000,...,63,84,83,86,90,11,11,14,8,11
219,G. Sigurðsson,29,186,77,Iceland,Everton,83,83,25500000,120000,...,79,84,57,57,55,13,8,10,15,12


In [10]:
X = a[['skill_ball_control','attacking_short_passing','skill_dribbling','dribbling','passing','mentality_vision','mentality_composure','shooting','movement_reactions']]
y = a["overall"]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df
#predictions
y_pred = regressor.predict(X_test)
#compare the predictions
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head(5)

,Actual,Predicted
1265,76,74.575051
3118,73,75.100342
942,78,77.847643
135,84,84.199972
2122,74,74.192373


In [13]:
#Evaluation of the algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.973270236314154
Mean Squared Error: 1.8667396130441591
Root Mean Squared Error: 1.3662867975078143
